In [2]:
import pandas as pd
import os
current_runs_location = "C:\\Users\\jalatorre\\Documents\\GitHub\\travel-model-one\\utilities\\NextGenFwys\\ModelRuns.xlsx"
current_runs_df = pd.read_excel(current_runs_location, sheet_name='all_runs')
current_runs_list = current_runs_df.loc[current_runs_df['status'] == 'current', 'directory']
runs = current_runs_list
# runs = ['2035_TM152_NGF_NP07_Path1a_06_SimpleTolls01',\
#     '2035_TM152_NGF_NP07_Path1b_02_SimpleTolls01',\
#     '2035_TM152_NGF_NP07_Path1b_03_SimpleTolls01',\
#     '2035_TM152_NGF_NP07_Path1b_01_LowestTolls03',\
#     '2035_TM152_NGF_NP07_Path1b_01_HighestTolls02',\
#     '2035_TM152_NGF_NP07_Path1b_01_UniformTolls03',\
#     '2035_TM152_NGF_NP07_Path2a_01',\
#     '2035_TM152_NGF_NP07_Path2a_02',\
#     '2035_TM152_NGF_NP07_Path2b_01',\
#     '2035_TM152_NGF_NP07_Path2b_02',\
#     '2035_TM152_NGF_NP07_Path3a_04',\
#     '2035_TM152_NGF_NP07_Path3a_05',\
#     '2035_TM152_NGF_NP07_Path3b_04',\
#     '2035_TM152_NGF_NP07_Path3b_05',\
#     '2035_TM152_NGF_NP07_Path4_02',\
#     '2035_TM152_NGF_NP07_Path4_03']
# runs = ['2035_TM152_FBP_Plus_24_rerunTM1.5.2.5']


for run in runs:
    
    tm_run_location = "L:\\Application\\Model_One\\NextGenFwys\\Scenarios\\" + run
    tm_runid = tm_run_location.split('\\')[-1]
    dest_file = tm_run_location+'/OUTPUT/core_summaries/' +"ODTravelTime_byModeTimeperiod_reduced_file.csv"
    source_file = tm_run_location+'/OUTPUT/core_summaries/ODTravelTime_byModeTimeperiodIncome.csv'

    # skip if it exists already
    if os.path.isfile(dest_file):
        print("    Destination file {} exists -- skipping".format(dest_file))
        continue
    # skip if source file doesn't exist
    if not os.path.isfile(source_file):
        print("   Source file {} does not exist -- skipping".format(source_file))
        continue
    
    tm_od_travel_times_df = pd.read_csv(source_file)
    # define origin destination pairs
    OD_pairs = [['OAKLAND','SAN FRANCISCO'],['VALLEJO','SAN FRANCISCO'],['ANTIOCH','SAN FRANCISCO'],['ANTIOCH','OAKLAND'],['SAN JOSE','SAN FRANCISCO'],['OAKLAND','PALO ALTO'],['OAKLAND','SAN JOSE'],['LIVERMORE','SAN JOSE'],['FAIRFIELD','DUBLIN'],['SANTA ROSA','SAN FRANCISCO']]
    # load lookup file for a city's TAZs
    taz_cities_df = pd.read_csv('L:\\Application\\Model_One\\NextGenFwys\\metrics\\Input Files\\taz_with_cities.csv')
    tm_od_tt_with_cities_df = tm_od_travel_times_df.merge(taz_cities_df, left_on='orig_taz', right_on='taz1454', how='left', suffixes = ["",'_orig']).merge(taz_cities_df, left_on='dest_taz', right_on='taz1454', how='left', suffixes = ["",'_dest'])

    tm_od_tt_with_cities_df = tm_od_tt_with_cities_df.loc[(tm_od_travel_times_df['timeperiod_label'].str.contains("AM Peak") == True)| \
                                                          (tm_od_travel_times_df['timeperiod_label'].str.contains("PM Peak") == True)|\
                                                          (tm_od_travel_times_df['timeperiod_label'].str.contains("Midday") == True)]
    tm_od_tt_with_cities_df = tm_od_tt_with_cities_df.drop(['incQ', 'incQ_label'], axis = 1)
    output_df = pd.DataFrame()

    for od in OD_pairs:
        temp = tm_od_tt_with_cities_df.loc[(tm_od_tt_with_cities_df['CITY'].str.contains(od[0]) == True)].loc[(tm_od_tt_with_cities_df['CITY_dest'].str.contains(od[1]) == True)]
        output_df = output_df.append(temp)
    output_df = output_df[['orig_taz','dest_taz','trip_mode','timeperiod_label','avg_travel_time_in_mins', 'num_trips']]
    output_df.to_csv(dest_file, float_format='%.5f', index=False) #, header=False
    print("Wrote {}".format(dest_file))

,orig_taz,dest_taz,trip_mode,timeperiod_label,num_trips,avg_travel_time_in_mins,taz1454,CITY,taz1454_dest,CITY_dest
0,1,1,1,AM Peak,2,6.02,1,SAN FRANCISCO,1,SAN FRANCISCO
2,1,1,1,Midday,4,6.07,1,SAN FRANCISCO,1,SAN FRANCISCO
3,1,1,1,Midday,2,6.07,1,SAN FRANCISCO,1,SAN FRANCISCO
4,1,1,3,AM Peak,2,6.02,1,SAN FRANCISCO,1,SAN FRANCISCO
5,1,1,3,Midday,12,6.07,1,SAN FRANCISCO,1,SAN FRANCISCO
...,...,...,...,...,...,...,...,...,...,...
6316464,1454,1454,8,PM Peak,10,20.65,1454,NaN,1454,NaN
6316465,1454,1454,8,PM Peak,14,20.65,1454,NaN,1454,NaN
6316466,1454,1454,8,PM Peak,8,20.65,1454,NaN,1454,NaN
6316467,1454,1454,8,PM Peak,8,20.65,1454,NaN,1454,NaN


Wrote L:\Application\Model_One\NextGenFwys\Scenarios\2035_TM152_FBP_Plus_24/OUTPUT/core_summaries/ODTravelTime_byModeTimeperiod_reduced_file.csv
